In [2]:
import pandas as pd
import numpy as np

In [4]:
df_rates = pd.read_csv('../data/rates_data_v4.csv')
df_rates

,Unnamed: 0,Symbol,Timestamp,liquidityRate_avg,variableBorrowRate_avg,utilizationRate_avg,stableBorrowRate_avg
0,0,DAI,2020-12-01 16:00:00,8.991636e-04,0.003996,0.083333,0.042083
1,1,DAI,2020-12-01 17:00:00,0.000000e+00,0.000000,0.250000,0.046250
2,2,DAI,2020-12-01 18:00:00,0.000000e+00,0.000000,0.250000,0.046250
3,3,DAI,2020-12-01 19:00:00,0.000000e+00,0.000000,0.250000,0.046250
4,4,DAI,2020-12-01 20:00:00,0.000000e+00,0.000000,0.250000,0.046250
...,...,...,...,...,...,...,...
2141197,2141197,FRAX,2024-10-24 02:00:00,5.555079e-07,0.038178,1.870334,0.000000
2141198,2141198,FRAX,2024-10-24 03:00:00,5.555079e-07,0.038178,1.870334,0.000000
2141199,2141199,FRAX,2024-10-24 04:00:00,5.555079e-07,0.038178,1.870334,0.000000
2141200,2141200,FRAX,2024-10-24 05:00:00,5.555079e-07,0.038178,1.870334,0.000000


In [10]:
rates_symbols = df_rates['Symbol'].unique()
rates_symbols = [x.lower() for x in rates_symbols]

In [14]:
import pandas as pd
import glob
import os 

path = 'prices/*.csv'
csv_files = glob.glob(path)
df_price = pd.concat([
    pd.read_csv(file).assign(symbol=os.path.splitext(os.path.basename(file))[0])
    for file in csv_files
], ignore_index=True)
print(df_price.columns)
print(len(df_price))


Index(['Unnamed: 0', 'timestamp', 'open', 'high', 'low', 'close', 'volume',
       'close_time', 'quote_asset_volume', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'symbol'],
      dtype='object')
1187631


In [18]:

prices_symbols = df_price['symbol'].unique()
prices_symbols = [x.replace('USDT','') for x in prices_symbols]
prices_symbols = [x.lower() for x in prices_symbols]
len(prices_symbols)

38

In [19]:
common_symbols = list(set(prices_symbols).intersection(rates_symbols))
print(len(common_symbols))
print(common_symbols)

22
['yfi', 'knc', 'ens', 'wbtc', 'ren', 'aave', 'crv', 'susd', 'mkr', 'enj', 'bal', 'dai', 'usdc', 'bat', '1inch', 'mana', 'tusd', 'zrx', 'link', 'snx', 'busd', 'uni']


In [21]:
df_rates['Symbol'] = df_rates['Symbol'].apply(lambda x : x.lower())
df_rates_filtered = df_rates[df_rates['Symbol'].isin(common_symbols)]
print(len(df_rates_filtered))
print(len(df_rates_filtered['Symbol'].unique()))
print(df_rates_filtered['Symbol'].unique())

720681
22
['dai' 'tusd' 'usdc' 'susd' 'bat' 'link' 'knc' 'mkr' 'mana' 'zrx' 'snx'
 'wbtc' 'busd' 'enj' 'ren' 'yfi' 'aave' 'uni' 'crv' 'bal' 'ens' '1inch']


### Lets now check for NA's and zeros

In [25]:
# Initialize an empty list to store the counts for each symbol as rows
zero_counts_list = []

# Loop through each unique symbol in the DataFrame
for symbol in df_rates_filtered['Symbol'].unique():
    # Filter the DataFrame for the current symbol
    df_symbol = df_rates_filtered[df_rates_filtered['Symbol'] == symbol]
    
    # Count the number of zeros in the specified columns for the current symbol
    zero_counts = {
        'Symbol': symbol,
        'liquidityRate_avg_zeros': (df_symbol['liquidityRate_avg'] == 0).sum(),
        'variableBorrowRate_avg_zeros': (df_symbol['variableBorrowRate_avg'] == 0).sum(),
        'utilizationRate_avg_zeros': (df_symbol['utilizationRate_avg'] == 0).sum(),
        'stableBorrowRate_avg_zeros': (df_symbol['stableBorrowRate_avg'] == 0).sum(),
        'total_rows': len(df_symbol)  # Total count of rows for the current symbol
    }
    
    # Append the counts as a row to the list
    zero_counts_list.append(zero_counts)

# Create a DataFrame from the list
zero_counts_df = pd.DataFrame(zero_counts_list)
zero_counts_df



,Symbol,liquidityRate_avg_zeros,variableBorrowRate_avg_zeros,utilizationRate_avg_zeros,stableBorrowRate_avg_zeros,total_rows
0,dai,2044,2081,0,0,34317
1,tusd,6440,6503,1,0,34222
2,usdc,404,412,2,0,34296
3,susd,6280,6282,1,34257,34257
4,bat,4979,4982,2,0,33158
5,link,1714,1734,6,0,34222
6,knc,5199,5256,2,0,32964
7,mkr,3374,3475,4,0,34297
8,mana,6431,6492,2,0,34146
9,zrx,5328,5373,2,0,33796


In [26]:
accpetable_tokens = ['bat', 'link', 'knc', 'mkr', 'mana', 'zrx', 'snx', 'wbtc', 'enj', 'ren', 'yfi', 'uni', 'crv', 'bal', 'ens', '1inch']
len(accpetable_tokens)

16

In [28]:
print(df_rates_filtered['variableBorrowRate_avg'].isna().sum())
print(df_rates_filtered['liquidityRate_avg'].isna().sum())

0
0
